**Supervised Learning 1**

# Linear Regression

## Part 2: Multiple Linear Regression

In this notebook, we'll expand our single-variate model to include multiple explanatory features.

Real-world policy outcomes rarely depend on a single factor. Let's build a more comprehensive model using multiple predictors.

In our example, income deprivation tells at least part of the story, but what about employment and child poverty? These factors often cluster together but may have independent effects.

<br>
<br>

> [📚 Scikit-learn Linear Models Documentation](https://scikit-learn.org/stable/modules/linear_model.html)





<br>

---

#### Setup: Import Libraries

In [1]:
import pandas as pd         # For data manipulation
import altair as alt        # For plotting our results
import numpy as np          # For numerical operations


## // Models
from sklearn import linear_model

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score


---

<br>
<br>

(We'll continue with the case study we saw in the previous notebook)

## Case Study: The Glasgow Effect 

The "Glasgow Effect" refers to the unexplained poor health outcomes in Glasgow compared to other UK cities, even after accounting for deprivation. Let's explore relationships between deprivation and life expectancy using real data from 61 Glasgow neighbourhoods.

**Load the data**

In [2]:
# Load the Glasgow health data (directly into a pandas dataframe)
data_url = 'https://raw.githubusercontent.com/RDeconomist/RDeconomist.github.io/main/charts/extreme/glasgowHealthData.csv'
data = pd.read_csv(data_url)
data.head()

,areaName,incomeDeprevation,employmentDeprivation,childPoverty,femaleLE,maleLE,disabilityRate
0,"Anniesland, Jordanhill and Whiteinch",0.14,0.15,0.14,80.8,75.8,0.19
1,Arden and Carnwadric,0.26,0.25,0.34,76.0,72.8,0.22
2,Baillieston and Garrowhill,0.12,0.12,0.14,81.6,76.0,0.21
3,Balornock and Barmulloch,0.29,0.27,0.38,78.2,70.8,0.30
4,"Bellahouston, Craigton and Mosspark",0.20,0.18,0.22,80.5,73.9,0.29


<br>
<br>

### Step 1. Explore the relationship

Let's use a loop with our basic chart code to plot the relationship between each of our explanatory variables.

In [3]:
feature_vars = ['incomeDeprevation', 'employmentDeprivation', 'childPoverty', 'disabilityRate']

In [9]:
charts = []
for variable in feature_vars:
    chart = alt.Chart(data).mark_point(color='rgba(128,0,0,.8)').encode(
        x=alt.X(f'{variable}:Q').axis(format='%').title(variable).scale(domain=[0, 0.55]),                    # On each iteration, we set the x-axis field and title to the current variable `variable`
        y=alt.Y('maleLE:Q').scale(zero=False, padding=40).title('Male life expectancy').axis(
            titleAngle=0, titleY=-2, titleAlign='left', titleX=1
        ),
    ).properties(width=250, height=250)
    charts.append(chart)

# Show horizontal concatenation of all charts
alt.hconcat(*charts)

alt.HConcatChart(...)

**What do we see?** strong negative relationship for income deprivation, employment deprivation, and child poverty and male life expectancy. Disability rate may have a slight negative relationship but it looks more unclear.

<br>
<br>
<br>

### Step 2: Prepare Training Data

As before, we need to extract our input and output data.

- **X** (uppercase): Feature matrix (can have multiple columns). E.g. Numpy array, numeric pandas DataFrame or Series
- **y** (lowercase): Target vector (single column of outcomes)


In [10]:
# Select multiple features
features = ['incomeDeprevation', 'employmentDeprivation', 'childPoverty', 'disabilityRate']

X_multi = data[features]        # Filter dataframe to only include columns in `features`
y = data['maleLE']

<br>
<br>
<br>

### Step 3. Fit the model

In [11]:
# Fit multiple regression
model_multi = linear_model.LinearRegression()
model_multi.fit(X_multi, y)         # Fit the model, this looks the same as with the single-variate model, but now we have multiple features in X

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


<br>
<br>
<br>

### Step 4. Inspect the results

In [16]:
print(f"Coefficients: {model_multi.coef_}")
print(f"Intercept: {model_multi.intercept_}")

Coefficients: [ 31.81011925 -32.56560719 -29.15891298   8.34715914]
Intercept: 79.26039195704499


Our `.coef_` array has multiple values, corresponding to each variable coefficient. Let's display this more clearly:

In [17]:
# Loop through features list and print their corresponding coefficients
for i in range(len(features)):  
    print(f"  {features[i]:25s}: {model_multi.coef_[i]:8.2f}")

  incomeDeprevation        :    31.81
  employmentDeprivation    :   -32.57
  childPoverty             :   -29.16
  disabilityRate           :     8.35


<br>

We can get the R-squared in the same way as before.

In [18]:
print(f"R-squared: {model_multi.score(X_multi, y):.3f}")

R-squared: 0.777


<br>
<br>
<br>

<br>
<br>

**How does this compare with our single linear regression?** (make sure the code in Section 1 has been run so `model_male` and `y_male` exist.)

In [19]:
# Quickly fit a single variate model
model_single = linear_model.LinearRegression()
X_single = data[['incomeDeprevation']]
model_single.fit(X_single, y)

# Print the results of each
print(f"Model Comparison:")
print(f"  Simple model R²: {model_single.score(X_single, y):.3f}")
print(f"  Multiple model R²: {model_multi.score(X_multi, y):.3f}")
print(f"  Improvement: {(model_multi.score(X_multi, y) - model_single.score(X_single, y)):.3f}")

Model Comparison:
  Simple model R²: 0.584
  Multiple model R²: 0.777
  Improvement: 0.193


So, our multi-variate model, with `employmentDeprivation`, `childPoverty` as well as `incomeDeprivation`, seems to capture more of the variance.

<br>
<br>
<br>

### Step 5. Visualisation (advanced)

Once we move beyond two dimensions in our analysis, visualising the model parameters and results can become more complicated. 

A helpful way to start is to think about what we are interested in. In this case, it would be useful to visualise the difference in prediction between our multi-variate and single-variate models.

In [20]:
# Generate predictions for both models, and add them back to our dataframe
data['pred_single'] = model_single.predict(X_single)
data['pred_multi'] = model_multi.predict(X_multi)

data.head(2)

,areaName,incomeDeprevation,employmentDeprivation,childPoverty,femaleLE,maleLE,disabilityRate,pred_single,pred_multi
0,"Anniesland, Jordanhill and Whiteinch",0.14,0.15,0.14,80.8,75.8,0.19,74.897865,76.332680
1,Arden and Carnwadric,0.26,0.25,0.34,76.0,72.8,0.22,71.176788,71.311966


<br>
<br>

**Option 1**: To visually compare the model predictions, we could plot the actual life expectancy against the value predicted by each model.

In [21]:
# Single variable model scatter
single_scatter = alt.Chart(data).mark_point(color='steelblue', opacity=0.6).encode(
    x=alt.X('pred_single:Q').title('Predicted life expectancy (single model)').scale(domain=[65, 82]),  # lock the domain scale to match across the axes
    y=alt.Y('maleLE:Q').title('Actual life expectancy').scale(domain=[65, 82]),
    tooltip=['areaName:N', alt.Tooltip('maleLE:Q', format='.1f', title='Actual'), alt.Tooltip('pred_single:Q', format='.1f', title='Predicted')]
).properties(
    title=f'Single variable model, with R² = {model_single.score(X_single, y):.1%}'        # Custom title with R-squared value
)


# Multiple variable model scatter  
multi_scatter = alt.Chart(data).mark_point(color='darkred', opacity=0.6).encode(
    x=alt.X('pred_multi:Q').title('Predicted life expectancy (multiple model)').scale(domain=[65, 82]),
    y=alt.Y('maleLE:Q').title('Actual life expectancy').scale(domain=[65, 82]),
    tooltip=['areaName:N', alt.Tooltip('maleLE:Q', format='.1f', title='Actual'), alt.Tooltip('pred_multi:Q', format='.1f', title='Predicted')]
).properties(
    title=f'Multiple variable model, with R² = {model_multi.score(X_multi, y):.1%}'
)


# Add a perfect prediction line (45-degree, so we can just add a point at each diagonal end)
perfect_line_data = pd.DataFrame({'x': [65, 82], 'y': [65, 82]})
perfect_line = alt.Chart(perfect_line_data).mark_line(
    strokeDash=[5, 5], color='gray'
).encode(x='x:Q', y='y:Q')


# Layer each chart with the 'perfect line', then concatenate them horizontally
(single_scatter + perfect_line) | (multi_scatter + perfect_line)      # Visualise both charts, horizontally concatenated

alt.HConcatChart(...)

<br>
<br>
<br>
<br>

**Option 2**: We could instead focus on the residuals, that is the difference between each actual and predicted value.

In [22]:
# Residual analysis - Check model assumptions
data['residual_single'] = y - data['pred_single']       # Calculate residuals
data['residual_multi'] = y - data['pred_multi']

data.head(3)

,areaName,incomeDeprevation,employmentDeprivation,childPoverty,femaleLE,maleLE,disabilityRate,pred_single,pred_multi,residual_single,residual_multi
0,"Anniesland, Jordanhill and Whiteinch",0.14,0.15,0.14,80.8,75.8,0.19,74.897865,76.332680,0.902135,-0.532680
1,Arden and Carnwadric,0.26,0.25,0.34,76.0,72.8,0.22,71.176788,71.311966,1.623212,1.488034
2,Baillieston and Garrowhill,0.12,0.12,0.14,81.6,76.0,0.21,75.518045,76.840389,0.481955,-0.840389


In [ ]:
# Single model residuals
single_resid = alt.Chart(data).mark_point(color='steelblue', opacity=0.6).encode(
    x=alt.X('pred_single:Q').title('Predicted life expectancy').scale(domain=[65, 82]),
    y=alt.Y('residual_single:Q').title('Residuals').scale(domain=[-10, 10]),
    tooltip=['areaName:N', alt.Tooltip('residual_single:Q', format='.2f')]
).properties(height=250, title='Residuals: Single Model')


# Multiple model residuals
multi_resid = alt.Chart(data).mark_point(color='darkred', opacity=0.6).encode(
    x=alt.X('pred_multi:Q').title('Predicted life expectancy').scale(domain=[65, 82]),
    y=alt.Y('residual_multi:Q').title('Residuals').scale(domain=[-10, 10]),
    tooltip=['areaName:N', alt.Tooltip('residual_multi:Q', format='.2f')]
).properties(height=250, title='Residuals: Multiple Model')


# Create zero line to add to each plot
zero_line = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(
    strokeDash=[3, 3], color='gray'
).encode(y='y:Q')


(single_resid + zero_line) | (multi_resid + zero_line)  # Add zero line to each plot, then concatenate them horizontally

alt.HConcatChart(...)

<br>

So our multiple variable model produces estimates with smaller errors from the true values.

We've seen how easy it is to move from single to multiple variable models

<br>

---

<br>

##### How to take this analysis further? 
- We've fitted some simple models, but can we trust the co-efficients? As with any econometric analysis, the interpretability of the model relies on some underlying assumptions holding. 
- One issue that we often see is **multicollinearity** - that is when our features are correlated with each other.
    - This can impact model stability, such that small changes in data can produce large changes in coefficients.

In the next notebook example, Linear Regression 3, we'll introduce some more advanced linear models that use **regularisation** techniques to address issues with multicollinearity and overfitting.
